In [1]:
import ijson
import zipfile
import pandas as pd

In [2]:
def extract_itens(input_file_path, target_inst=None):
    set_of_itens = []

    with zipfile.ZipFile(input_file_path, "r") as zip_file:
        filename= zip_file.namelist()[0]
    
        with zip_file.open(filename) as json_file:
            parser = ijson.parse(json_file)

        # Inicie o parsing do JSON
            in_docs = False
            for prefix, event, value in parser:
                if in_docs:
                    if event == 'start_map':
                        doc = {}  # Inicie um novo dicionário para cada documento "docs"
                    elif event == 'end_map':
                    # Faça algo com o documento, por exemplo, imprimir ou processar
                    # i+=1
                    # if i > limit:
                    #     break
                        if target_inst!= None:
                            try:
                                if target_inst.get(doc['instacron_str']) != None:
                                    set_of_itens.append(doc)
                            except KeyError:
                                pass
                        else:
                            set_of_itens.append(doc)
                    else:
                        short_prexif = prefix.split('.')[-1]
                        if short_prexif != 'item':
                            doc[short_prexif] = value
                elif prefix == 'response.docs':
                    in_docs = True

# Certifique-se de fechar o arquivo após a leitura
    json_file.close()
    return set_of_itens

- id - o id do documento no Oasisbr
- oai_identifier_str - id externo que vem do repositório de origem
- instname_str - nome da instituição de origem
- instacron_str - acrônimo da instituição de origem
- network_acronym_str - acrônimo da rede de origem (pode descartar)
- network_name_str - nome da rede origem (pode descartar)
- title - título
- author - autores
- publishDate  - data da publicação
- format - tipo de documento (article, masterThesis, doctoralThesis, …)
- description - abstract
- topic - keywords
- language - idioma do documento
- eu_rights_str_mv - tipo de acesso (openAccess, closed, …)
- url - url externa para o documento

In [3]:
input_file_path="D:\\tmp\oaisis\oasisbr_5.zip"

target_inst = {
                'UFCG' : 1,
               'UEPB' : 1
            }

pd.DataFrame(extract_itens(input_file_path, target_inst=target_inst)).head(2)

,id,oai_identifier_str,network_acronym_str,network_name_str,title,author,topic,description,publishDate,format,url,language,eu_rights_str_mv,instname_str,instacron_str,docs
0,UFCG_06f953de5632423215f2902ee1c9977f,oai:localhost:riufcg/6033,UFCG,Biblioteca Digital de Teses e Dissertações da ...,O uso da iconografia no ensino de história: um...,None,NaN,Esta pesquisa centra-se no estudo da iconograf...,None,None,None,NaN,None,Universidade Federal de Campina Grande (UFCG),UFCG,NaN
1,UFCG_1f3af7882a45950655b4fb799ea6aa22,oai:localhost:riufcg/6034,UFCG,Biblioteca Digital de Teses e Dissertações da ...,Usos sociais da escrita por alunos de turmas d...,None,NaN,A presente investigação tem como foco a anális...,None,None,None,NaN,None,Universidade Federal de Campina Grande (UFCG),UFCG,NaN


In [10]:
input_path="D:\\tmp\oaisis\\"
file_list = ['oasisbr_1.zip','oasisbr_2.zip','oasisbr_3.zip','oasisbr_4.zip','oasisbr_5.zip']

target_inst = {
               'UEPB' : 1
}

output=[]

for i in file_list:
    input_file_path = input_path + i
    out_local = extract_itens(input_file_path, target_inst=target_inst)
    output = output + out_local



NameError: name 'extract_itens' is not defined

In [6]:
uepb = pd.DataFrame(output)

In [9]:
uepb.network_name_str.value_counts()

network_name_str
Biblioteca Digital de Teses e Dissertações da UEPB    2388
Revista de Estudos Internacionais (REI)                 11
Repositório Institucional da UEPB                        2
Name: count, dtype: int64

# doi query

In [9]:
import sys
sys.path.append("../")
from clients import CrossrefClient

titulo_publicacao = "Incremental Entity Blocking over Heterogeneous Streaming Data"
titulo_publicacao = "Towards automatic Privacy-Preserving Record Linkage: A Transfer Learning based classification step"
cr_client = CrossrefClient()
cr_client.get_doi_by_title(titulo_publicacao)


'10.1016/j.datak.2023.102180'

# oai query

http://harvester.ibict.br:8092/request?verb=GetRecord&identifier=oai:periodicos.ufsc.br:article/26431&metadataPrefix=xoai


```
http://harvester.ibict.br:8092/request?verb=GetRecord&identifier= + id + &metadataPrefix=xoai
```

In [ ]:
import requests

# URL do servidor OAI
oai_base_url = 'http://example.com/oai-pmh-server'

# Identificador que você deseja recuperar
identifier = 'oai:periodicos.ufsc.br:article/26431'

# Parâmetros da consulta
params = {
    'verb': 'GetRecord',  # A ação para recuperar um registro
    'metadataPrefix': 'oai_dc',  # O formato de metadados desejado
    'identifier': identifier  # O identificador único do registro
}

# Realizar a solicitação HTTP
response = requests.get(oai_base_url, params=params)

if response.status_code == 200:
    # O servidor respondeu com sucesso, e você pode acessar os metadados da resposta
    oai_metadata = response.content  # Isso contém os metadados do registro no formato especificado (neste caso, Dublin Core)

    # Agora você pode processar os metadados como desejado
    print(oai_metadata)
else:
    # Lidar com possíveis erros
    print("Erro na solicitação OAI-PMH. Código de status:", response.status_code)
    print(response.text)  # Isso pode conter informações de erro do servidor

In [ ]:
user_to_repos = {}

with zipfile.ZipFile("./data/drug-event-Q4-0001-of-0013.json.zip", "r") as z:


with open("large-file.json", "rb") as f:
    for record in ijson.items(f, "item"):
        user = record["actor"]["login"]
        repo = record["repo"]["name"]
        if user not in user_to_repos:
            user_to_repos[user] = set()
        user_to_repos[user].add(repo)